In [0]:
!python --version
!pip --version

In [0]:
!ls

In [0]:
!mv sample_data GeneratingFaces

In [0]:
!rm GeneratingFaces/*

In [0]:
!cd GeneratingFaces
!git clone https://github.com/toufeeqahamedns/GeneratingFaces.git

In [0]:
!pip install --no-cache-dir -r GeneratingFaces/requirements.txt
!pip install -q --upgrade ipython==5.5.0
!pip install -q --upgrade ipykernel==4.6.0

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
!python --version
!pip --version

In [0]:
!pip install https://download.pytorch.org/whl/cu100/torch-1.0.0-cp36-cp36m-linux_x86_64.whl
# !pip install https://download.pytorch.org/whl/cu100/torch-1.0.0-cp27-cp27m-linux_x86_64.whl
# !pip install https://download.pytorch.org/whl/cu100/torch-1.0.0-cp27-cp27mu-linux_x86_64.whl
# !pip install torch==0.4.0

In [0]:
!python --version
!pip --version

In [0]:
!mkdir /content/GeneratingFaces/implementation/networks/InferSent/models

In [0]:
!cp /content/gdrive/My\ Drive/GeneratingFaces/infersent2.pkl /content/GeneratingFaces/implementation/networks/InferSent/models/infersent2.pkl
#!curl -Lo /content/GeneratingFaces/implementation/networks/InferSent/models/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

In [0]:
!ls -lah /content/GeneratingFaces/implementation/networks/InferSent/models/infersent2.pkl

In [0]:
# v1
# !wget http://nlp.stanford.edu/data/glove.840B.300d.zip
# !unzip glove.840B.300d.zip

#!mv /content/gdrive/My\ Drive/GeneratingFaces/glove.840B.300d.txt /content/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt

In [0]:
# v1
#!mv glove.840B.300d.txt /content/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt

In [0]:
# v1
#!cp /content/gdrive/My\ Drive/GeneratingFaces/glove.840B.300d.txt /content/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt

In [0]:
# v1
#!ls -lah /content/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt

In [0]:
# v1
# !cp /content/gdrive/My\ Drive/GeneratingFaces/glove.840B.300d.txt /content/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt

In [0]:
# For v2 require once
# !cp /content/gdrive/My\ Drive/GeneratingFaces/fasttext*.zip /content/GeneratingFaces/
# !unzip /content/GeneratingFaces/fasttext*.zip

In [0]:
# For v2 require once
# !cp crawl-300d-2M.vec /content/gdrive/My\ Drive/GeneratingFaces/crawl-300d-2M.vec
# !mv crawl-300d-2M.vec /content/GeneratingFaces/implementation/networks/InferSent/models/crawl-300d-2M.vec

In [0]:
!cp /content/gdrive/My\ Drive/GeneratingFaces/crawl-300d-2M.vec /content/GeneratingFaces/implementation/networks/InferSent/models/crawl-300d-2M.vec

In [0]:
# For v2
!ls -lah /content/GeneratingFaces/implementation/networks/InferSent/models/crawl*.vec

In [0]:
!python --version
!pip --version

In [0]:
# Not needed for v2
# !ls -lah /content/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt

In [0]:
# Not needed for v2
# !wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
# !tar -xf lfw.tgz

In [0]:
!cp /content/gdrive/My\ Drive/GeneratingFaces/celeba-dataset.zip /content/

In [0]:
!mkdir /content/dump
!unzip /content/celeba-dataset.zip -d /content/dump

In [0]:
!mv /content/dump/img_align_celeba/img_align_celeba /content/GeneratingFaces/celeba
!rm -r /content/dump

In [0]:
!nvcc --version
!python --version
!pip --version

In [1]:
import sys
sys.path.append('/home/toufeeq/CollegeProject/GeneratingFaces/implementation')
sys.path.append('/home/toufeeq/CollegeProject/GeneratingFaces/')

In [2]:
from train_network import main as main_train
from train_network import get_config

In [3]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--config", action="store", type=str, default="/home/toufeeq/CollegeProject/GeneratingFaces/implementation/configs/2_colab.conf",
                    help="default configuration for the Network")
parser.add_argument("--start_depth", action="store", type=int, default=0,
                    help="Starting depth for training the network")
parser.add_argument("--encoder_file", action="store", type=str, default=None,
                    help="pretrained Encoder file (compatible with my code)")
parser.add_argument("--ca_file", action="store", type=str, default=None,
                    help="pretrained Conditioning Augmentor file (compatible with my code)")
parser.add_argument("--generator_file", action="store", type=str, default=None,
                    help="pretrained Generator file (compatible with my code)")
parser.add_argument("--discriminator_file", action="store", type=str, default=None,
                    help="pretrained Discriminator file (compatible with my code)")

_StoreAction(option_strings=['--discriminator_file'], dest='discriminator_file', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help='pretrained Discriminator file (compatible with my code)', metavar=None)

In [4]:
!python --version
!pip --version

Python 3.6.10 :: Anaconda, Inc.
pip 20.0.2 from /home/toufeeq/anaconda3/envs/GeneratingFaces/lib/python3.6/site-packages/pip (python 3.6)


In [5]:
args = parser.parse_args([])

In [6]:
config = get_config(args.config)

In [7]:
import torch as th

class ConditionAugmentor(th.nn.Module):
    """ Perform conditioning augmentation
        from the paper -> https://arxiv.org/abs/1710.10916 (StackGAN++)
        uses the reparameterization trick from VAE paper.
    """

    def __init__(self, input_size, latent_size, use_eql=True, device=th.device("cpu")):
        """
        constructor of the class
        :param input_size: input size to the augmentor
        :param latent_size: required output size
        :param use_eql: boolean for whether to use equalized learning rate
        :param device: device on which to run the Module
        """
        super(ConditionAugmentor, self).__init__()

        assert latent_size % 2 == 0, "Latent manifold has odd number of dimensions"

        # state of the object
        self.device = device
        self.input_size = input_size
        self.latent_size = latent_size

        # required modules:
        if use_eql:
            from pro_gan_pytorch.CustomLayers import _equalized_linear
            self.transformer = _equalized_linear(self.input_size, 2 * self.latent_size).to(device)
        else:
            self.transformer = th.nn.Linear(self.input_size, 2 * self.latent_size).to(device)

    def forward(self, x, epsilon=1e-12):
        """
        forward pass (computations)
        :param x: input
        :param epsilon: a small noise added for numerical stability
        :return: c_not_hat, mus, sigmas => augmented text embeddings, means, stds
        """
        from torch.nn.functional import relu

        # apply the feed forward layer:
        combined = self.transformer(x)

        # use the reparameterization trick
        mid_point = self.latent_size
        mus, sigmas = combined[:, :mid_point], combined[:, mid_point:]

        # mus don't need to be transformed, but sigmas cannot be negative.
        # so, we'll apply a ReLU on top of sigmas
        sigmas = relu(sigmas)  # hopefully the network will learn a good sigma mapping
        sigmas = sigmas + epsilon  # small noise added for stability

        epsilon = th.randn(*mus.shape).to(self.device)
        c_not_hat = (epsilon * sigmas) + mus

        return c_not_hat, mus, sigmas


In [8]:
!python --version
!pip --version

Python 3.6.10 :: Anaconda, Inc.
pip 20.0.2 from /home/toufeeq/anaconda3/envs/GeneratingFaces/lib/python3.6/site-packages/pip (python 3.6)


In [9]:
device = 'cuda'

import data_processing.DataLoader as dl

dataset = dl.RawTextFace2TextDataset(
    annots_file=config.annotations_file,
    img_dir=config.images_dir,
    img_transform=dl.get_transform(config.img_dims)
)
from networks.TextEncoder import PretrainedEncoder
# create a new session object for the pretrained encoder:
text_encoder = PretrainedEncoder(
    model_file=config.pretrained_encoder_file,
    embedding_file=config.pretrained_embedding_file,
    device=device
)

Vocab size : 300000


In [10]:
condition_augmenter = ConditionAugmentor(
    input_size=config.hidden_size,
    latent_size=config.ca_out_size,
    use_eql=config.use_eql,
    device='cuda'
)

In [11]:
temp_data = dl.get_data_loader(dataset, 64, num_workers=3)
fixed_captions, fixed_real_images = iter(temp_data).next()
fixed_embeddings = text_encoder(fixed_captions)
fixed_embeddings.shape

(64, 4096)

In [12]:
condition_augmenter.train()

# create fixed_input for debugging
temp_data = dl.get_data_loader(dataset, 64, num_workers=3)
fixed_captions, fixed_real_images = iter(temp_data).next()
fixed_embeddings = text_encoder(fixed_captions)
fixed_embeddings = th.from_numpy(fixed_embeddings).to(device)

fixed_c_not_hats, _, _ = condition_augmenter(fixed_embeddings)

fixed_noise = th.randn(len(fixed_captions),
                       config.latent_size - fixed_c_not_hats.shape[-1]).to(device)

fixed_gan_input = th.cat((fixed_c_not_hats, fixed_noise), dim=-1)

In [13]:
fixed_embeddings.shape

torch.Size([64, 4096])

In [14]:
import torch
torch.load('/home/toufeeq/CollegeProject/GeneratingFaces/implementation/networks/InferSent/models/infersent2.pkl')

OrderedDict([('enc_lstm.weight_ih_l0',
              tensor([[-0.0188, -0.0096, -0.0165,  ...,  0.0063,  0.0177,  0.0182],
                      [ 0.0040, -0.0170,  0.0282,  ...,  0.0270, -0.0006, -0.0099],
                      [-0.0333,  0.0013,  0.0235,  ...,  0.0239, -0.0444,  0.0009],
                      ...,
                      [ 0.0068, -0.0130, -0.0136,  ..., -0.0003,  0.0145, -0.0031],
                      [-0.0075, -0.0210,  0.0135,  ...,  0.0149, -0.0059,  0.0018],
                      [-0.0076,  0.0241, -0.0258,  ..., -0.0104,  0.0177,  0.0241]])),
             ('enc_lstm.weight_hh_l0',
              tensor([[ 2.4178e-02, -3.4815e-03, -1.5027e-02,  ...,  2.3643e-02,
                       -4.6200e-03, -1.2944e-02],
                      [ 2.1155e-02, -9.4947e-03,  2.8347e-02,  ..., -1.1898e-02,
                       -1.6273e-02, -1.7331e-02],
                      [ 1.6204e-02, -2.0330e-02, -6.8777e-03,  ..., -8.8209e-03,
                       -1.4847e-02, -1.4684e-

In [15]:
from networks.TextEncoder import PretrainedEncoder
# create a new session object for the pretrained encoder:
text_encoder = PretrainedEncoder(
    model_file='/home/toufeeq/CollegeProject/GeneratingFaces/implementation/networks/InferSent/models/infersent2.pkl',
    embedding_file='/home/toufeeq/CollegeProject/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt',
    device='cuda'
)

Vocab size : 300000


In [16]:
main_train(args)

/home/toufeeq/CollegeProject/GeneratingFaces/implementation/configs/2_colab.conf
Current Configuration: {'images_dir': '/home/toufeeq/CollegeProject/GeneratingFaces/celeba', 'processed_text_file': '/home/toufeeq/CollegeProject/GeneratingFaces/implementation/processed_annotations/processed_text_celeba.pkl', 'annotations_file': '/home/toufeeq/CollegeProject/GeneratingFaces/data/face2text_v1.0/raw.json', 'pretrained_encoder_file': '/home/toufeeq/CollegeProject/GeneratingFaces/implementation/networks/InferSent/models/infersent2.pkl', 'pretrained_embedding_file': '/home/toufeeq/CollegeProject/GeneratingFaces/implementation/networks/InferSent/models/glove.840B.300d.txt', 'log_dir': '/home/toufeeq/CollegeProject/GeneratingFaces/training_runs/3/losses/', 'sample_dir': '/home/toufeeq/CollegeProject/GeneratingFaces/training_runs/3/generated_samples/', 'save_dir': '/home/toufeeq/CollegeProject/GeneratingFaces/training_runs/3/saved_models/', 'captions_length': 100, 'img_dims': [64, 64], 'use_pretr

KeyboardInterrupt: 

In [0]:
import pandas as pd

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
for i in range(4, 5):
  log = pd.read_csv('/content/gdrive/My Drive/GeneratingFaces/training_runs/3/losses/loss_{}.log'.format(i), delimiter='\t', header=None, names=['d_loss', 'g_loss', 'kl_loss'])
  n = len(log.d_loss)
  xs = list(range(n*i, n*(i + 1)))
  plt.plot(xs, log.d_loss)
  plt.plot(xs, log.g_loss)
  plt.plot(xs, log.kl_loss)
  plt.legend(log.columns)

In [0]:
plt.plot(log.d_loss)
plt.plot(log.g_loss)
plt.plot(log.kl_loss)
plt.legend(log.columns)

In [0]:
from networks.ConditionAugmentation import ConditionAugmentor
from pro_gan_pytorch.PRO_GAN import ConditionalProGAN

import numpy as np

def create_grid(samples, scale_factor, img_file, real_imgs=False):
    """
    utility function to create a grid of GAN samples
    :param samples: generated samples for storing
    :param scale_factor: factor for upscaling the image
    :param img_file: name of file to write
    :param real_imgs: turn off the scaling of images
    :return: None (saves a file)
    """
    from torchvision.utils import save_image
    from torch.nn.functional import interpolate

    samples = th.clamp((samples / 2) + 0.5, min=0, max=1)

    # upsample the image
    if not real_imgs and scale_factor > 1:
        samples = interpolate(samples,
                              scale_factor=scale_factor)

    # save the images:
    save_image(samples, img_file, nrow=int(np.sqrt(len(samples))))

In [0]:
# create the networks

condition_augmenter = ConditionAugmentor(
    input_size=config.hidden_size,
    latent_size=config.ca_out_size,
    use_eql=config.use_eql,
    device=device
)

ca_file = '/content/gdrive/My Drive/GeneratingFaces/training_runs/3/saved_models/Condition_Augmentor_4.pth'

print("Loading conditioning augmenter from:", ca_file)
condition_augmenter.load_state_dict(th.load(ca_file))

c_pro_gan = ConditionalProGAN(
    embedding_size=config.hidden_size,
    depth=config.depth,
    latent_size=config.latent_size,
    compressed_latent_size=config.compressed_latent_size,
    learning_rate=config.learning_rate,
    beta_1=config.beta_1,
    beta_2=config.beta_2,
    eps=config.eps,
    drift=config.drift,
    n_critic=config.n_critic,
    use_eql=config.use_eql,
    loss=config.loss_function,
    use_ema=config.use_ema,
    ema_decay=config.ema_decay,
    device=device
)

generator_file = '/content/gdrive/My Drive/GeneratingFaces/training_runs/3/saved_models/GAN_GEN_4.pth'
print("Loading generator from:", generator_file)
c_pro_gan.gen.load_state_dict(th.load(generator_file))

In [0]:
condition_augmenter.train(False)

In [0]:
temp_data = dl.get_data_loader(dataset, 1, num_workers=3)
fixed_captions, fixed_real_images = iter(temp_data).next()

In [0]:
fixed_embeddings = text_encoder(fixed_captions)
fixed_embeddings = th.from_numpy(fixed_embeddings).to(device)

fixed_c_not_hats, mus, _ = condition_augmenter(fixed_embeddings)

fixed_noise = th.zeros(len(fixed_captions),
                       c_pro_gan.latent_size - fixed_c_not_hats.shape[-1]).to(device)

fixed_gan_input = th.cat((fixed_c_not_hats, fixed_noise), dim=-1)

In [0]:
create_grid(
    samples=c_pro_gan.gen(
        fixed_gan_input,
        4,
        1.0
    ),
    scale_factor=1,
    img_file='output.png')

img = plt.imread('output.png')
plt.figure()
plt.imshow(img)

In [0]:
fixed_captions

In [0]:
fixed_embeddings.shape